In [1]:
import pandas as pd
import numpy as np
import time
import nltk
nltk.download('punkt')
from transformers import *
import matplotlib.pyplot as plt
import torch
import json
from seqeval.metrics import classification_report,f1_score
from seqeval.scheme import IOB2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ken19980727\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df_kai = pd.read_csv('mention_medications_tweets_smm4h2018_roberta_predicted.tsv',sep = '\t')
df_me = pd.read_csv('Non-Process_data/BioCreative_ValTask3.tsv',sep = '\t')
df_kai

,text,predicted
0,Why do I get so emotional when I take adderall,1
1,RT @saranghaemeng: @Black_SheepPH Need my anxi...,1
2,Someone give them the coughing drops!,1
3,Woman accidentally prescribed erectile dysfunc...,1
4,RT @fshakir: Sanders to join diabetes patients...,1
...,...,...
10368,This is a friendly reminder to get your flu sh...,1
10369,Is it possible to build up an immunity to thin...,1
10370,Have I taken too much allergy medication or ar...,1
10371,Does Your Dog Really Need The Kennel Cough Vac...,1


In [3]:
df_kai[df_kai['predicted'] == 1]

,text,predicted
0,Why do I get so emotional when I take adderall,1
1,RT @saranghaemeng: @Black_SheepPH Need my anxi...,1
2,Someone give them the coughing drops!,1
3,Woman accidentally prescribed erectile dysfunc...,1
4,RT @fshakir: Sanders to join diabetes patients...,1
...,...,...
10368,This is a friendly reminder to get your flu sh...,1
10369,Is it possible to build up an immunity to thin...,1
10370,Have I taken too much allergy medication or ar...,1
10371,Does Your Dog Really Need The Kennel Cough Vac...,1


In [4]:
model_ty = 'bert-base-cased'
# model_ty = 'bert-base-multilingual-cased'
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained(model_ty, do_lower_case=False)

In [5]:
MODEL = torch.load('Model/BERT_case_BC.pt')
with open('Model/tag_map.json') as jsonfile:
    tag2idx = json.load(jsonfile)
MODEL.load_state_dict(torch.load(f'bert-base-cased.pt'))
MODEL.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [6]:
idx2tag = {tag2idx[t]:t for t in tag2idx}
idx2tag

{0: 'I-', 1: 'O', 2: 'B-', 3: 'PAD'}

In [7]:
def predict(model,sent,o_sent):
    torch_sent = torch.LongTensor(torch.tensor([sent])).cuda()
    with torch.no_grad():
        output = model(torch_sent)[0][0]
        out_tag = [idx2tag[o] for o in torch.max(output,1)[1].cpu().numpy()]
    pred = []
    for s,t in zip(o_sent,out_tag):
        if s == '<ETY>':
            pred.append(t)
    return pred

In [8]:
sentences_pred = []
tags_pred = []
def first_wordpicee(w_p):
    w_p[0] = '<ETY>'
    return w_p
    
for i in df_kai.index:
    tokens = nltk.word_tokenize(df_kai['text'][i])
    SENT = []
    O_SENT = []
    for token in tokens:
        O_SENT.extend(first_wordpicee(tokenizer.tokenize(token)))
        SENT.extend(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(token)))
    try:
        tag = predict(MODEL,SENT,O_SENT)
        tags_pred.append(tag)
        sentences_pred.append(tokens)
    except:
        print(SENT)

[8329, 2007, 11135, 7418, 1132, 2318, 1106, 1129, 1678, 1111, 2551, 1808, 1111, 170, 1148, 2004, 1104, 7560, 1105, 1111, 119, 119, 119, 18630, 131, 120, 120, 189, 119, 1884, 120, 171, 2559, 4880, 3361, 4426, 1665, 2087, 1559, 23161, 122, 155, 1942, 137, 153, 5821, 1732, 2249, 17540, 131, 22327, 21902, 9872, 117, 2001, 20192, 27230, 12812, 17627, 2577, 21327, 1158, 12120, 21484, 2692, 3273, 18630, 131, 120, 120, 189, 119, 1884, 120, 144, 5082, 1942, 5301, 1197, 13821, 2591, 4538, 122, 155, 1942, 137, 170, 14467, 4063, 5084, 15540, 131, 19680, 131, 1263, 8750, 12473, 1164, 12030, 1105, 3485, 1654, 1105, 23006, 13987, 119, 122, 120, 122, 137, 8050, 1811, 9238, 168, 137, 189, 2246, 1204, 168, 180, 25534, 14964, 8752, 1179, 3291, 1116, 1197, 1775, 1822, 20149, 1105, 174, 15776, 4854, 5250, 170, 1665, 1673, 4044, 6906, 122, 137, 1112, 17162, 13689, 1348, 20695, 1986, 1738, 117, 5194, 27825, 1106, 11769, 2660, 7540, 117, 1159, 1106, 1243, 1149, 106, 122, 23009, 1104, 159, 1174, 11014, 10337, 

In [9]:
len(sentences_pred)

10372

In [10]:
data_train = pd.read_csv("IOB2_Data/BC_train_IOB2_all.txt",sep = '\t', na_filter=False)
data_dev = pd.read_csv("IOB2_Data/BC_dev_IOB2_all.txt",sep = '\t', na_filter=False)
# data_train = data_train.fillna('NA')
# data_dev = data_dev.fillna('NA')

In [11]:
data_dev

,Word,Tag,Length,Sentence#
0,I,O,1,0
1,'m,O,2,0
2,37,O,2,0
3,weeks,O,5,0
4,pregnant,O,8,0
...,...,...,...,...
548096,another,O,7,37897
548097,day,O,3,37897
548098,I,O,1,37897
548099,'m,O,2,37897


In [12]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [13]:
getter = SentenceGetter(data_train)
dev_getter = SentenceGetter(data_dev)

In [14]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
dev_sentences = [[word[0] for word in sentence] for sentence in dev_getter.sentences]

In [15]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
dev_labels = [[s[1] for s in sent] for sent in dev_getter.sentences]

In [16]:
sentences.extend(sentences_pred)
labels.extend(tags_pred)

In [17]:
tag_values = list(set(data_train["Tag"].values))
tag_values.append("PAD")
tag_values

['I-', 'B-', 'O', 'PAD']

In [18]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)
        if n_subwords > 1 and 'B-' in label:
            labels.extend([label])
            _ = 'I-' + label.split('B-')[1]
            if _ not in tag_values:
                tag_values.append(_)
                print(_)
            labels.extend([_] * (n_subwords-1))
        else:
            labels.extend([label] * n_subwords)
    return tokenized_sentence, labels

In [19]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]
print('done')
dev_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(dev_sentences, dev_labels)
]

done


In [20]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
tokenized_labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

dev_tokenized_texts = [token_label_pair[0] for token_label_pair in dev_tokenized_texts_and_labels]
dev_tokenized_labels = [token_label_pair[1] for token_label_pair in dev_tokenized_texts_and_labels]

In [21]:
def check_post_entity(pre_tag,tag):
    error_entity = False
    if pre_tag == 'O' and 'I-' in tag:
        error_entity = True
    if error_entity:
        print(idx+1)
        print(pre_tag,tag)
def inspect_label(lal):
    for n,_ in enumerate(lal):
        if n != 0:
            check_post_entity(lal[n-1],lal[n])
for idx,t_label in enumerate(dev_tokenized_labels):
    inspect_label(t_label)

In [22]:
input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
dev_input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in dev_tokenized_texts]
tags = [[tag2idx.get(l) for l in lab] for lab in tokenized_labels]
dev_tags = [[tag2idx.get(l,tag2idx['O']) for l in lab] for lab in dev_tokenized_labels]

In [23]:
from torch.utils.data import Dataset
class BertNerDataset(Dataset):
    def __init__(self,sentences,labels, word_pad_idx, tag_pad_idx, max_len = 500):
        self.sentences = sentences
        self.labels = labels
        self.word_pad_idx = word_pad_idx
        self.tag_pad_idx = tag_pad_idx
        self.max_len = max_len
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        return (self.sentences[index],self.labels[index])
        
    def collate_fn(self, datasets):
        sentences = [dataset[0] for dataset in datasets]
        labels = [dataset[1] for dataset in datasets]
        max_sent = max([len(data) for data in sentences])
        max_len = max([min(len(sentence), self.max_len) for sentence in sentences])
        pad_sentence = []
        pad_label = []
        for sentence,label in zip(sentences,labels):
            
            if len(sentence) > max_len:
#                 print('asd')
                pad_sentence.append(sentence[:max_len])
                pad_label.append(label[:max_len])
                attention_masks = [[float(i != 0.0) for i in ii] for ii in pad_sentence]
            else:
#                 print('zxc')
                pad_sentence.append(sentence+[self.word_pad_idx]*(max_len-len(sentence)))
                pad_label.append(label+[self.tag_pad_idx]*(max_len-len(label)))
                attention_masks = [[float(i != 0.0) for i in ii] for ii in pad_sentence]
        return torch.LongTensor(pad_sentence), torch.LongTensor(pad_label),torch.tensor(attention_masks)

In [24]:
tokenizer.convert_tokens_to_ids('[PAD]')

0

In [25]:
from torch.utils.data import TensorDataset, DataLoader , SubsetRandomSampler ,ConcatDataset,Sampler
from transformers import get_linear_schedule_with_warmup
bs = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 5

tr_dataset = BertNerDataset(input_ids,tags,tokenizer.convert_tokens_to_ids('[PAD]'),tag2idx['PAD'])
va_dataset = BertNerDataset(dev_input_ids,dev_tags,tokenizer.convert_tokens_to_ids('[PAD]'),tag2idx['PAD'])

In [26]:
from tqdm import tqdm,trange

model = BertForTokenClassification.from_pretrained(
    model_ty,
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)
optimizer = AdamW(model.parameters(),lr=3e-5,eps=1e-8)
train_dataloader = DataLoader(tr_dataset, batch_size=bs,
                            collate_fn=tr_dataset.collate_fn)
valid_dataloader = DataLoader(va_dataset, batch_size=bs,
                            collate_fn=va_dataset.collate_fn)
all_loader = {
    "train" : train_dataloader,
    "valid" : valid_dataloader,
}
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
    )
model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [27]:
F_Score = []
for _ in trange(10, desc="Epoch"):
    for loader in all_loader:
        total_loss = 0
        # Training loop
        predictions , true_labels = [], []
        for step, batch in enumerate(all_loader[loader]):
            if loader == 'train':
                model.train()
            else:
                model.eval()
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_labels ,b_input_mask = batch
            model.zero_grad()
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)

            loss = outputs[0]
            loss.backward()
            total_loss += loss.item()
            if loader == 'train' or 1:
#                 torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
                optimizer.step()
                scheduler.step()
            logits = outputs[1].detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.extend(label_ids)

        avg_loss = total_loss / len(all_loader[loader])
        print(f"Average {loader} loss: {avg_loss}")
        pred_tags,valid_tags = [],[]
        for p, l in zip(predictions, true_labels):
            _p = []
            _l = []
            for p_i, l_i in zip(p, l):
                if tag_values[l_i] != "PAD":
                    _p.append(idx2tag[p_i])
                    _l.append(idx2tag[l_i])
            pred_tags.append(_p)
            valid_tags.append(_l)
        print(f"{loader} F1-Score: {f1_score(valid_tags , pred_tags,scheme = IOB2)}")
        if loader == 'valid':
            F_Score.append(f1_score(valid_tags , pred_tags,scheme = IOB2))
            try:
                print(classification_report(valid_tags , pred_tags ,mode='strict' ,scheme = IOB2))
            except:
                pass

Epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]

Average train loss: 0.007243064785808355


C:\ProgramData\Anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


train F1-Score: 0.537661922277307
Average valid loss: 0.0009670018322239871
valid F1-Score: 0.6051282051282051


Epoch:  10%|███████▌                                                                   | 1/10 [05:25<48:51, 325.77s/it]

...

              precision    recall  f1-score   support

           _       0.98      0.99      0.99       105

   micro avg       0.98      0.99      0.99       105
   macro avg       0.98      0.99      0.99       105
weighted avg       0.98      0.99      0.99       105

Average train loss: 0.0007587704978466529
train F1-Score: 0.8974609375
Average valid loss: 3.4167876308181245e-05
valid F1-Score: 0.9719626168224299


Epoch:  90%|███████████████████████████████████████████████████████████████████▌       | 9/10 [48:54<05:26, 326.63s/it]

              precision    recall  f1-score   support

           _       0.98      0.99      0.99       105

   micro avg       0.98      0.99      0.99       105
   macro avg       0.98      0.99      0.99       105
weighted avg       0.98      0.99      0.99       105

Average train loss: 0.0007713263346208687
train F1-Score: 0.892891918208374
Average valid loss: 3.4167876308181245e-05
valid F1-Score: 0.9719626168224299


Epoch: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [54:20<00:00, 326.04s/it]

              precision    recall  f1-score   support

           _       0.98      0.99      0.99       105

   micro avg       0.98      0.99      0.99       105
   macro avg       0.98      0.99      0.99       105
weighted avg       0.98      0.99      0.99       105



In [28]:
np.round(np.array(F_Score),2)

array([0.61, 0.89, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97, 0.97])

In [29]:
set([v for vs in pred_tags for v in vs])

{'B-', 'I-', 'O'}

In [34]:
print(classification_report(valid_tags , pred_tags ,mode='strict' ,scheme = IOB2))

              precision    recall  f1-score   support

           _       0.75      0.55      0.64       105

   micro avg       0.75      0.55      0.64       105
   macro avg       0.75      0.55      0.64       105
weighted avg       0.75      0.55      0.64       105



## Testset

In [30]:
import nltk
nltk.download('punkt')

df_test = pd.read_csv('Non-process_data/BioCreative_TEST_Task3_PARTICIPANTS.tsv',encoding = 'utf-8',sep = '\t')
df_test[39584:39585]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ken19980727\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,tweet_id,user_id,created_at,text,start,end,span,drug
39584,601134480515080193,454565608,2015-05-20,A S D,-,-,-,-


In [31]:
def process_test(df_test):
    t_sents = []
    for i in df_test.index:
        tokens = nltk.word_tokenize(df_test['text'][i].strip())
        t_sents.append(tokens)
    return t_sents

test_sentences = process_test(df_test)

In [32]:
pred_valid_num = []
def tokenize_word_process(sentence):
    tokenized_sentence = []
    for word in sentence:
        tokenized_word = tokenizer.tokenize(word)
        tokenized_sentence.extend(tokenized_word)
    num_tokenize_word = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    num_tokenize_word = torch.tensor([num_tokenize_word]).cuda()
    return num_tokenize_word

for i in test_sentences:
    pred_valid_num.append(tokenize_word_process(i))

In [33]:
all_label_indices = []
c = 0
for i in pred_valid_num:
    with torch.no_grad():
        try:
            output = model(i)
        except:
            print(len(i[0]))
            print(c)
            print('')
            cls = True
    c += 1
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    all_label_indices.append(label_indices)

In [34]:
all_new_labels = []
for i in range(len(pred_valid_num)):
    new_labels = []
    for label_idx in all_label_indices[i][0]:
            new_labels.append(tag_values[label_idx])
    all_new_labels.append(new_labels)
all_new_labels

[['B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-'],
 ['B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-'],
 ['B-', 'B-', 'B-', 'B-', 'B-', 'B-', 'B-'],
 ['B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-',
  'B-'],
 ['B-', 'B-', 'B-', 'B-', 'B-', 'B-'],
 ['B-',
  'B-',
 

In [35]:
def tokenize_and_preserve_words(sentence):
    tokenized_sentence = []
    
    for word in sentence:
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        if n_subwords > 1:
            for idx,_ in enumerate(tokenized_word):
                if idx != 0 and '##' not in tokenized_word[idx]:
                    tokenized_word[idx] = '##' + tokenized_word[idx]
        tokenized_sentence.extend(tokenized_word)
    return tokenized_sentence

pred_valid_word = []
for i in test_sentences:
    pred_valid_word.append(tokenize_and_preserve_words(i))
pred_valid_word

[['Dick',
  'me',
  'over',
  'there',
  'and',
  'go',
  'to',
  'the',
  'hospital',
  'go',
  'to',
  'the',
  'hospital',
  'go',
  'to',
  'the',
  'hospital',
  'https',
  ':',
  '/',
  '##/',
  '##t',
  '##.',
  '##co',
  '##/',
  '##0',
  '##X',
  '##W',
  '##uad',
  '##gt',
  '##Y',
  '##z'],
 ['CA',
  '##MI',
  '+',
  'MI',
  '##DI',
  'SK',
  '##IR',
  '##T',
  '/',
  '##/',
  'W',
  '##H',
  '##IT',
  '##E',
  'T',
  '##EE',
  '+',
  'L',
  '##EA',
  '##TH',
  '##ER',
  'L',
  '##EG',
  '##GI',
  '##NG',
  '##S',
  'https',
  ':',
  '/',
  '##/',
  '##t',
  '##.',
  '##co',
  '##/',
  '##3',
  '##Y',
  '##J',
  '##a',
  '##G',
  '##y',
  '##7',
  '##M',
  '##8',
  '##A',
  'on',
  '@',
  'blog',
  '##lov',
  '##in'],
 ['@', 'j', '##ti', '##mber', '##lake', 'the', 'legend'],
 ['I',
  'think',
  'my',
  'baby',
  'do',
  'n',
  "##'",
  '##t',
  'like',
  'the',
  'doctors',
  'because',
  'I',
  'was',
  'throwing',
  'up',
  'this',
  'morning',
  'and',
  'the',
  'bug',
 

In [36]:
dev_predition = []

for sent_,label_ in zip(pred_valid_word,all_new_labels):
    res = False
    al_label = []
    for i,(word,label) in enumerate(zip(sent_,label_)):
        if '##' in word:
            continue
        al_label.append(label)
    dev_predition.append(al_label)

In [37]:
data_test = {}
data_test['Word'] = []
data_test['Tag'] = []
data_test['Length'] = []
data_test['Sentence#'] = []

c = 0
for sent,pred in zip(test_sentences,dev_predition):
    for s,p in zip(sent,pred):
        data_test['Word'].append(s)
        data_test['Tag'].append(p)
        data_test['Length'].append(len(s))
        data_test['Sentence#'].append(c)
    c += 1
# data_test

In [38]:
data_test = pd.DataFrame(data_test)
data_test

,Word,Tag,Length,Sentence#
0,Dick,B-,4,0
1,me,B-,2,0
2,over,B-,4,0
3,there,B-,5,0
4,and,B-,3,0
...,...,...,...,...
785979,tips,B-,4,54481
785980,!,B-,1,54481
785981,?,B-,1,54481
785982,😂😂,B-,2,54481


In [39]:
data_test.to_csv('Bert data_argumentation.csv',index = None )

In [51]:
asd = pd.read_csv('test(Bert data_argumentation).csv'
                  , na_filter=False
                 )
asd

,Word,Tag,Length,Sentence#
0,Dick,O,4,0
1,me,O,2,0
2,over,O,4,0
3,there,O,5,0
4,and,O,3,0
...,...,...,...,...
785979,tips,O,4,54481
785980,!,O,1,54481
785981,?,O,1,54481
785982,😂😂,O,2,54481
